This notebook is to train and store models to disc.\
This Notebook has to be clean (do not define functions here, do them in an
external utils.py and import them).\
This notebook has to be reproducible (if you run it twice, the same output has to
be displayed and stored to disk).

In [2]:
# import libraries and set randome seed for reproducibility
import pandas as pd
import numpy as np
import sklearn
import scipy
from sklearn import *
import os
RANDOM_SEED = 123 # taken from task description

In [4]:
# use this to train and VALIDATE your solution
train_df = pd.read_csv("./quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./quora_test_data.csv")

In [5]:
# divide into train, val, test split
A_df, te_df = sklearn.model_selection.train_test_split(train_df, test_size=0.05, random_state=RANDOM_SEED)
tr_df, va_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05, random_state=RANDOM_SEED)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


Simple solution: 